In [1]:
import pandas as pd
from datetime import datetime, timedelta
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [2]:
def scroll_down(driver):
    # 페이지 맨 아래로 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(7)  # 스크롤 후 잠시 대기

def get_news_info(driver, query):
    # 기사 정보 추출
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    news_items = soup.find_all('div', class_='news_area')
    
    news_info = []
    for item in news_items:
        # 기사 제목 추출
        title_elem = item.find('a', class_='news_tit')
        title = title_elem.text.strip() if title_elem else None
        
        # 제목에 검색어가 포함되어 있지 않은 경우 크롤링하지 않음
        if not title or query not in title:
            continue

        # 기사 날짜 추출
        date_elems = item.find_all('span', class_='info')
        date = None
        for date_elem in date_elems:
            if date_elem.find('i', class_='spnew ico_paper'):
                continue  # i 태그가 있는 경우 건너뛰기
            # 날짜 형식에 맞는 텍스트 확인
            if re.match(r'\d{4}\.\d{2}\.\d{2}', date_elem.text.strip()):
                date = date_elem.text.strip()
                break
        
        if date:
            news_info.append({'제목': title, '날짜': date})
    
    return news_info

# 검색어 입력
query = input("뉴스 검색어를 입력하세요: ")

# 날짜 설정
start_date = datetime(2021, 6, 1)
end_date = datetime(2024, 5, 31)

# WebDriver로 Chrome 브라우저 열기
driver = webdriver.Chrome()

# 빈 리스트 생성
data = []

# 각 달마다 검색 수행
current_date = start_date
while current_date <= end_date:
    # 현재 달의 시작일과 종료일 계산
    next_month = current_date + timedelta(days=31)
    next_month = next_month.replace(day=1)  # 다음 달의 1일로 설정
    end_of_month = next_month - timedelta(days=1)  # 다음 달의 마지막 날짜로 설정
    
    # 네이버 뉴스 검색 페이지로 이동
    driver.get(f"https://search.naver.com/search.naver?where=news&query={query}&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds={current_date.strftime('%Y.%m.%d')}&de={end_of_month.strftime('%Y.%m.%d')}")
    
    # 스크롤 다운을 100번 반복하여 데이터 가져오기 - 달 별로 1000개 
    for i in range(100): 
        scroll_down(driver)
        
    # 스크롤 다운이 완료된 후 데이터 가져오기
    news_info = get_news_info(driver, query)
    data.extend(news_info)
    
    # 다음 달로 이동
    current_date = next_month

# 브라우저 닫기
driver.quit()

# 데이터프레임 생성
df = pd.DataFrame(data)

# '제목' 열에서 중복 제거
df = df.drop_duplicates(subset='제목').reset_index(drop=True)

# 데이터프레임 출력
print(df)

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.185)
Stacktrace:
0   chromedriver                        0x0000000104832a0c chromedriver + 4385292
1   chromedriver                        0x000000010482b318 chromedriver + 4354840
2   chromedriver                        0x0000000104448b0c chromedriver + 281356
3   chromedriver                        0x000000010443215c chromedriver + 188764
4   chromedriver                        0x0000000104432098 chromedriver + 188568
5   chromedriver                        0x00000001044c3454 chromedriver + 783444
6   chromedriver                        0x000000010447feec chromedriver + 507628
7   chromedriver                        0x00000001044808c4 chromedriver + 510148
8   chromedriver                        0x00000001047fa3c8 chromedriver + 4154312
9   chromedriver                        0x00000001047fee2c chrome

In [ ]:
df.to_csv('./dataset/sk_new.csv', index=False)